In [2]:
!pip install torch

In [3]:
import json
import pandas as pd
import torch
from tqdm.notebook import tqdm

In [4]:
vidpath = '/Users/pauli/Downloads/CS_396_DS_pipeline/amazon_reviews_us_Video.tsv'
df_vid = pd.read_csv(vidpath, sep='\t', quoting = 3)

In [5]:
df_vid.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,49033728,R1P1G5KZ05H6RD,6302503213,748506413,The Night They Saved Christmas [VHS],Video,5,0,0,N,Y,Very satisfied!!,Fast shipping. Pleasure to deal with. Would re...,2015-08-31
1,US,17857748,R106N066IUN8ZV,B000059PET,478710180,"Hamlet / Kline, New York Shakespeare Festival ...",Video,5,0,0,N,Y,The most talented actor ever!,"Kevin Kline is the most versatile, multi-talen...",2015-08-31
2,US,25551507,R7WTAA1S5O7D9,0788812807,981002815,Nascar Dual Powered Calculator (Solar & Batter...,Video,4,0,0,N,Y,Four Stars,great movie,2015-08-31
3,US,21025041,R32HFMVWLYOYJK,6302509939,333219811,"The Man From U.N.C.L.E, Volume 19: The Brain K...",Video,5,0,0,N,Y,Five Stars,i love the martin landau episode,2015-08-31
4,US,40943563,RWT3H6HBVAL6G,B00JENS2BI,538101194,Playboy Video Party Jokes [VHS],Video,3,0,0,N,N,"HOT women, dumb jokes",Y'know what this reminded me of? Those sketch ...,2015-08-31


In [6]:
#New column- star rating 3+ is positive, 2 or less is negative (or 4+?)
target = pd.DataFrame([1 if x > 3 else 0 for x in df_vid['star_rating']])
target.head()

,0
0,1
1,1
2,1
3,1
4,0


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_vid,
                                                  target,
                                                  test_size=0.2)



In [8]:
!pip install transformers

In [9]:
!pip install typing-extensions --upgrade

In [10]:

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df_vid[df_vid.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_vid[df_vid.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_vid[df_vid.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)